In [8]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [9]:
# 1. 아이리스 데이터 로드 + 판다스 DataFrame 변환
iris = load_iris()
X = iris.data                          # (150, 4) 특징(feature)
y = iris.target                        # (150,) 레이블(label: 0,1,2)

df = pd.DataFrame(X, columns=iris.feature_names) #컬럼이름에 피쳐네임 지정
df["target"] = y #예측변수 지정

print(df.head())






   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


In [11]:
# 2. 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    df[iris.feature_names],
    df["target"],
    test_size=0.2,
    random_state=42,
    stratify=df["target"], #타겟을 기준으로 비율 지정
)


In [ ]:
# 3. 스케일링(Standardization)
scaler = StandardScaler() #scaler에 스케일링함수 저장
X_train_scaled = scaler.fit_transform(X_train) #X_train에 평균,편차 학습후 변환
X_test_scaled = scaler.transform(X_test) #X_test 테스트에 스케일링 변환



In [ ]:
# 4. 딥러닝 모델 정의 (입력 4차원 데이터, 3층 신경망 은닉층 2개, 출력1개 3클래스)
model = keras.Sequential([ #keras.Sequential 모델 지정
    layers.Input(shape=(4,)),          # 꽃받침/꽃잎 길이·너비 4개 특성 인풋값에 특성4개 입력
    layers.Dense(16, activation="relu"), #히든레이어 1
    layers.Dense(16, activation="relu"), #히든레이어 2
    layers.Dense(3, activation="softmax")  # 출력층 클래스 3개
])#Dense는 뉴런
#차원 수에따라 히든레이어 지정후 1개는 출력층으로 지정

2025-11-28 15:47:27.635455: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-11-28 15:47:27.635515: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-11-28 15:47:27.635531: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-11-28 15:47:27.635582: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-28 15:47:27.635599: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:

# 5. 모델 컴파일
model.compile(
    optimizer="adam",                      # 옵티마이저(Optimizer) Adam 경사하강법 방법
    loss="sparse_categorical_crossentropy",# 손실 함수(Loss) – 정수 인코딩 스칼라값 손실함수.다중클래스 / 
    metrics=["accuracy"]                   # 정확도(Accuracy) 측정
)

# 6. EarlyStopping 콜백(과적합 방지)
early_stop = keras.callbacks.EarlyStopping( #keras모델이 검증로스 기준으로 epoch마다 재 계산
    monitor="val_loss", #과적합 감지를 검증로스로 보겠다.
    patience=10, #개선되지않는 에폭 횟수 20번나오면 멈춰라. 
    restore_best_weights=True #그중 성능이 가장좋았던모델 선택
)





In [15]:
# 7. 모델 학습
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,      # 학습 데이터의 20%를 검증(Validation)에 사용
    epochs=100, #200번 반복
    batch_size=6, # 전체 데이터를 /16으로 나눠서 
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100


2025-11-28 15:55:38.261754: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.6042 - loss: 0.7706 - val_accuracy: 0.5833 - val_loss: 0.7306
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7396 - loss: 0.5991 - val_accuracy: 0.5833 - val_loss: 0.6182
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8021 - loss: 0.5090 - val_accuracy: 0.7083 - val_loss: 0.5518
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8333 - loss: 0.4521 - val_accuracy: 0.7500 - val_loss: 0.5068
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8542 - loss: 0.4164 - val_accuracy: 0.7917 - val_loss: 0.4704
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8646 - loss: 0.3886 - val_accuracy: 0.7917 - val_loss: 0.4435
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8646 - loss: 0.3683 - val_accuracy: 0.8333 - val_loss: 0.4180
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8750 - loss: 0.3521 - val_accuracy: 0.8750 - val_l

In [16]:
# 8. 테스트 데이터로 평가
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Test Loss: 0.0711, Test Accuracy: 0.9667


In [17]:

# 9. 예측(Prediction) 예시
sample = X_test_scaled[:5]
pred_prob = model.predict(sample)          # 각 클래스 확률
pred_class = np.argmax(pred_prob, axis=1)  # 확률이 가장 큰 클래스 선택

print("실제 레이블:", y_test.to_numpy()[:5])
print("예측 레이블:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
실제 레이블: [0 2 1 1 0]
예측 레이블: [0 2 1 1 0]


---

# 파이토치

In [20]:
# 1. 데이터 준비
# scikit-learn에서 Iris 데이터셋 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


iris = load_iris()
X = iris.data
y = iris.target

# 데이터 표준화 (Normalization)
# 신경망 학습 성능 향상을 위해 각 특성의 평균을 0, 분산을 1로 만듭니다.
scaler = StandardScaler()
X = scaler.fit_transform(X) #동일하게 표준편차 기준 평균 학습후 변환

# 학습(train) 데이터와 테스트(test) 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# NumPy 배열을 PyTorch 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long) #int 64 로옹~
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 텐서 데이터셋 및 데이터로더 생성
#NumPy 배열인 데이터를 PyTorch가 처리할 수 있는 torch.tensor로 변환하고, 
# 이를 TensorDataset과 DataLoader로 묶습니다. DataLoader는 데이터를 
# 미니배치(mini-batch) 단위로 묶어 효율적으로 모델에 전달하는 역할을 합니다.

train_dataset = TensorDataset(X_train_tensor, y_train_tensor) #넘파이로 데이터 전달
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True) #총 데이터셋을 16으로 나눔 150/16 번
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 2. 모델 정의 (다층 퍼셉트론, MLP)
# Iris 데이터는 4개의 특성(feature)을 가지므로 입력 레이어는 4개 노드,
# 3개 클래스로 분류하므로 출력 레이어는 3개 노드를 가집니다.
#IrisClassifier는 nn.Module을 상속받아 신경망 모델을 정의합니다

class IrisClassifier(nn.Module): #아이리스 분류모델 클래스 지정.
    def __init__(self):
        super(IrisClassifier, self).__init__() #부모 생성자 호출하기
        self.fc1 = nn.Linear(4, 16)  # 입력(4) -> 은닉층(16)
        self.relu = nn.ReLU() #활성화 함수 > 이후 선형으로 분류
        self.fc2 = nn.Linear(16, 16) # 은닉층(16) -> 은닉층(16)
        self.fc3 = nn.Linear(16, 3)  # 은닉층(16) -> 출력(3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)  #소프트맥스함수 필요 없음
        return x #3종류의 클래스점수 로짓

# 모델, 손실 함수, 옵티마이저 초기화
model = IrisClassifier()
criterion = nn.CrossEntropyLoss() #소프트맥스,크로스엔트로피 교차 적용
"""
nn.CrossEntropyLoss는 다음과 같은 기능을 하나로 묶어 제공합니다.

소프트맥스(Softmax): 모델의 최종 출력값(로짓, logit)을 
각 클래스에 대한 확률 분포로 변환합니다.
로그(Log): 확률값에 로그를 취합니다.
음의 로그 가능도(Negative Log Likelihood) 손실 계산: 정답 클래스에 대한 확률의 로그값에 음수를 취해 손실을 계산합니다.
"""
optimizer = optim.Adam(model.parameters(), lr=0.01)#경사하강법 아담로직

# 3. 모델 학습
def train_model(epochs):
    for epoch in range(epochs):
        for inputs, labels in train_loader: #트레인 데이터셋에 대하여 인풋과 라벨을지정.
            optimizer.zero_grad()  # 그라디언트 초기화
            outputs = model(inputs) # 순전파 모델은 클래스
            loss = criterion(outputs, labels) # 손실 계산 크로스엔트로로피로
            loss.backward() # 역전파
            optimizer.step() # 가중치 업데이트
        
        # 에포크마다 손실 출력
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    print('학습 완료!')

# 4. 모델 평가 (학습 데이터셋 포함)
def evaluate_model():
    model.eval()  # 모델을 평가 모드로 전환
    with torch.no_grad(): # 그라디언트 계산 비활성화
        
        # 학습 데이터셋 정확도 계산
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:  #train_loader 안에서 
            outputs = model(inputs) #모델 입력값
            _, predicted = torch.max(outputs.data, 1) #_언더스코어 최댓값버리고 predicted 가장 큰 예측값저장.
            total_train += labels.size(0) #훈련수
            correct_train += (predicted == labels).sum().item() #모델 인풋값을 꺼내서  예측시 라벨과 맞는값일경우 전부 더해서  correct_train에 저장
        
        accuracy_train = 100 * correct_train / total_train #비율
        print(f'훈련 데이터셋 정확도: {accuracy_train:.2f}%')
        
        # 테스트 데이터셋 정확도 계산
        correct_test = 0
        total_test = 0
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
        
        accuracy_test = 100 * correct_test / total_test
        print(f'테스트 데이터셋 정확도: {accuracy_test:.2f}%')

# 학습 및 평가 실행
if __name__ == '__main__':
    train_model(epochs=100)
    evaluate_model()

Epoch [1/100], Loss: 0.8732
Epoch [2/100], Loss: 0.4511
Epoch [3/100], Loss: 0.4772
Epoch [4/100], Loss: 0.2857
Epoch [5/100], Loss: 0.3230
Epoch [6/100], Loss: 0.2374
Epoch [7/100], Loss: 0.0931
Epoch [8/100], Loss: 0.0773
Epoch [9/100], Loss: 0.3841
Epoch [10/100], Loss: 0.0527
Epoch [11/100], Loss: 0.2242
Epoch [12/100], Loss: 0.0287
Epoch [13/100], Loss: 0.0029
Epoch [14/100], Loss: 0.0140
Epoch [15/100], Loss: 0.3733
Epoch [16/100], Loss: 0.0501
Epoch [17/100], Loss: 0.0175
Epoch [18/100], Loss: 0.1803
Epoch [19/100], Loss: 0.1651
Epoch [20/100], Loss: 0.0736
Epoch [21/100], Loss: 0.0002
Epoch [22/100], Loss: 0.0009
Epoch [23/100], Loss: 0.0417
Epoch [24/100], Loss: 0.0007
Epoch [25/100], Loss: 0.4121
Epoch [26/100], Loss: 0.0613
Epoch [27/100], Loss: 0.0096
Epoch [28/100], Loss: 0.0039
Epoch [29/100], Loss: 0.0684
Epoch [30/100], Loss: 0.4786
Epoch [31/100], Loss: 0.1932
Epoch [32/100], Loss: 0.3487
Epoch [33/100], Loss: 0.0068
Epoch [34/100], Loss: 0.1021
Epoch [35/100], Loss: 0